# **Keras的掩盖和填充**

### **引入**

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### **介绍**

**掩盖**是一种通知序列处理层在输入中，某些时间步丢失的方法，因此在处理数据时应将其跳过。

**填充**是掩盖的一种特殊形式，位于掩盖步骤的开始或者序列的开始。填充是用于将序列数据编码为连续的批处理：为了使批处理中的所有序列都能达到标准长度，所以有必要填充或截断某些序列。

接下来，让我们详细讨论。

### **填充序列数据**
处理序列数据时，每个样本具有不同的长度是很常见的。思考以下示例（标记为单词的文本）：

In [ ]:
[
  ["Hello", "world", "!"],
  ["How", "are", "you", "doing", "today"],
  ["The", "weather", "will", "be", "nice", "tomorrow"],
]

在进行单词检索之后，数据可能被矢量化为整数，例如：

In [ ]:
[
  [71, 1331, 4231]
  [73, 8, 3215, 55, 927],
  [83, 91, 1, 645, 1253, 927],
]

数据是一个嵌套列表，其中各个样本的长度分别为3、5和6，由于深度学习模型的输入数据必须是单个张量（例如，在这种情况下形状应为`(batch_size, 6, vocab_size)`），因此，较短的样本需要用一些占位符值填充（当然，也可以在填充之前截断长样本）。

Keras提供了一个实用工具方法，可将Python列表截断并填充到相同的长度：[`tf.keras.preprocessing.sequence.pad_sequences`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)。

In [2]:
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

# 默认情况下，会使用0进行填充;你可以通过"value"属性进行配置
# 你可以使用"pre"填充(在序列前填充)或者使用"post"填充(在序列尾填充)
# 当使用RNN层时，我们推荐使用"post"填充(为了能够使用这些层的CuDNN实现)
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)
print(padded_inputs)

[[ 711  632   71    0    0    0]
 [  73    8 3215   55  927    0]
 [  83   91    1  645 1253  927]]


### **掩盖**

现在所有样本都具有统一的长度，必须告知模型该数据的某些部分实际上是填充的，应该忽略。这种机制正是**掩盖**。

在Keras模型中，有三种方法可以引入输入掩盖：

+ 添加一个[`keras.layers.Masking`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Masking)图层。
+ 使用配置了`mask_zero=True`的[`keras.layers.Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding)层。
+ 在支持`mask`参数的层中，手动传递该参数（例如RNN层）。

### **掩盖生成层：Embedding和Masking**

这些层将创建一个掩盖张量（一个shape为`(batch, sequence_length)`的2D张量），并将其附加到`Masking`或`Embedding`层返回的张量输出上。

In [3]:
embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)

print(masked_output._keras_mask)

masking_layer = layers.Masking()
# 通过将2D输入扩展为3D（嵌入尺寸为10）来模拟嵌入查找。
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
)

masked_embedding = masking_layer(unmasked_embedding)
print(masked_embedding._keras_mask)

tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)
tf.Tensor(
[[ True  True  True False False False]
 [ True  True  True  True  True False]
 [ True  True  True  True  True  True]], shape=(3, 6), dtype=bool)


从打印结果中可以看到，掩盖是shape为`(batch_size, sequence_length)`的2D布尔张量 ，其中每个False表示在处理过程中应忽略相应的时间步长。

### **函数式API和Sequential API中的掩盖传播**

使用函数式API或Sequential API时，由`Embedding`或`Masking`层生成的掩盖将通过网络传播到能够使用它们的层（例如，RNN层）。Keras将自动获取与输入相对应的掩盖，并将其传递给知道如何使用该掩盖的层。

例如，在下面的Sequential模型中，LSTM层将自动接收掩盖，这意味着它将忽略填充值：

In [ ]:
model = keras.Sequential(
    [layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True), layers.LSTM(32),]
)

以下函数式API模型也是如此：

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

### **将掩盖张量直接传递到层**

可以处理掩盖的层（例如`LSTM`层）在其`__call__`方法中具有一个`mask`参数。

同时，产生掩盖的层（例如Embedding）会暴露`compute_mask(input, previous_mask)`方法，这样你可以调用该方法。

因此，你可以将掩盖生成层的`compute_mask(`)方法的输出，传递给掩盖处理层的`__call__`方法，如下所示：

In [4]:
class MyLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(MyLayer, self).__init__(**kwargs)
        self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
        self.lstm = layers.LSTM(32)

    def call(self, inputs):
        x = self.embedding(inputs)
        # 请注意，你也可以手动准备“mask”张量
        # 它只需要是一个具有正确形状的布尔张量，即（batch_size，timesteps）
        mask = self.embedding.compute_mask(inputs)
        output = self.lstm(x, mask=mask)  # 层将会忽略掩盖值
        return output


layer = MyLayer()
x = np.random.random((32, 10)) * 100
x = x.astype("int32")
layer(x)

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[-2.9557333e-03, -4.9592513e-03, -5.5697327e-03, ...,
        -6.4263460e-03,  1.7222196e-03, -5.9980587e-03],
       [ 1.2630115e-03,  6.7719403e-03, -2.2096483e-06, ...,
        -1.5865450e-06, -2.0998481e-03,  1.5725335e-03],
       [-5.8411937e-03,  1.5372749e-03,  8.2000103e-03, ...,
        -1.2324885e-03, -3.1526345e-03,  4.1902601e-04],
       ...,
       [-1.0838141e-03,  5.6212828e-03,  2.4681108e-03, ...,
         9.0770936e-03, -4.8025837e-03,  8.2275616e-03],
       [-5.9154308e-03,  2.6111908e-03, -5.9640841e-03, ...,
        -1.1013175e-02, -2.0490203e-04, -3.4524666e-03],
       [ 4.0665409e-03, -1.0666442e-03, -3.6945881e-03, ...,
        -5.4598167e-03,  7.1055471e-04, -6.4708097e-03]], dtype=float32)>

### **在你自定义层中支持掩盖**

有时，你可能需要编写生成掩盖的层（例如`Embedding`），或需要修改当前掩盖的层。

例如，任何产生张量的时间维度与其输入不同的层，例如以时间维度串联的`Concatenate`层，都需要修改当前掩盖，以便下游层能够理解掩盖的时间步长。

为此，你的层应实现该`layer.compute_mask()`方法，该方法将根据输入和当前掩盖生成一个新的掩盖。

下面是一个`TemporalSplit`的示例，该层需要修改当前掩盖。

In [5]:
class TemporalSplit(keras.layers.Layer):
    """Split the input tensor into 2 tensors along the time dimension."""

    def call(self, inputs):
        # 预期输入为3D，掩盖为2D，沿时间轴（轴1）将输入张量分成2个子张量
        return tf.split(inputs, 2, axis=1)

    def compute_mask(self, inputs, mask=None):
        # 如果存在的话，也将掩盖分成2个
        if mask is None:
            return None
        return tf.split(mask, 2, axis=1)


first_half, second_half = TemporalSplit()(masked_embedding)
print(first_half._keras_mask)
print(second_half._keras_mask)

tf.Tensor(
[[ True  True  True]
 [ True  True  True]
 [ True  True  True]], shape=(3, 3), dtype=bool)
tf.Tensor(
[[False False False]
 [ True  True False]
 [ True  True  True]], shape=(3, 3), dtype=bool)


下面是`CustomEmbedding`的一个示例，该层能够根据输入值生成掩盖：

In [6]:
class CustomEmbedding(keras.layers.Layer):
    def __init__(self, input_dim, output_dim, mask_zero=False, **kwargs):
        super(CustomEmbedding, self).__init__(**kwargs)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.mask_zero = mask_zero

    def build(self, input_shape):
        self.embeddings = self.add_weight(
            shape=(self.input_dim, self.output_dim),
            initializer="random_normal",
            dtype="float32",
        )

    def call(self, inputs):
        return tf.nn.embedding_lookup(self.embeddings, inputs)

    def compute_mask(self, inputs, mask=None):
        if not self.mask_zero:
            return None
        return tf.not_equal(inputs, 0)


layer = CustomEmbedding(10, 32, mask_zero=True)
x = np.random.random((3, 10)) * 9
x = x.astype("int32")

y = layer(x)
mask = layer.compute_mask(x)

print(mask)

tf.Tensor(
[[ True  True  True  True  True False  True  True  True  True]
 [ True  True  True  True  True  True  True  True False  True]
 [False  True  True False  True  True  True  True  True  True]], shape=(3, 10), dtype=bool)


### **选择性使用兼容层上的掩盖传播**

大多数层都不会修改时间维度，因此不需要修改当前掩盖。但是，他们可能仍然希望能够将当前的掩盖保持不变地**传播**到下一层。**这是一种选择性行为**，默认情况下，自定义层将破坏当前的掩盖（因为框架无法确定传播该掩盖是否安全）。

如果你有一个不修改时间维度的自定义层，并且希望它能够传播当前输入掩盖，则应在层构造函数中设置`self.supports_masking = True`。在这种情况下，的默认行为`compute_mask()`是仅将当前掩盖传递下去。

下面的示例展示将掩盖传播列入白名单的层：

In [7]:
class MyActivation(keras.layers.Layer):
    def __init__(self, **kwargs):
        super(MyActivation, self).__init__(**kwargs)
        # 表示该层可安全进行掩盖传播
        self.supports_masking = True

    def call(self, inputs):
        return tf.nn.relu(inputs)

现在，你可以在生成掩盖的层（如`Embedding`）和使用掩盖的层（如`LSTM`）之间使用此自定义层，它将传递掩盖，使其到达使用掩盖的层。

In [8]:
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
x = MyActivation()(x)  # 会直接传递掩盖
print("Mask found:", x._keras_mask)
outputs = layers.LSTM(32)(x)  # 将会接收到掩盖

model = keras.Model(inputs, outputs)

Mask found: Tensor("embedding_2/NotEqual:0", shape=(None, None), dtype=bool)


### **编写需要掩盖信息的层**

有些层是掩盖的使用者：它们在`call`方法中接受一个`mask`参数，并用它来确定是否跳过某些时间步长。

要编写这样的层，你只需在`call`方法定义中，添加一个`mask=None`参数即可。只要有，与输入关联的掩盖将被传递到你的层。

下面是一个简单的示例：一个层，该层在输入序列的时间维度（轴1）上计算softmax，同时丢弃掩盖的时间步长。

In [9]:
class TemporalSoftmax(keras.layers.Layer):
    def call(self, inputs, mask=None):
        broadcast_float_mask = tf.expand_dims(tf.cast(mask, "float32"), -1)
        inputs_exp = tf.exp(inputs) * broadcast_float_mask
        inputs_sum = tf.reduce_sum(inputs * broadcast_float_mask, axis=1, keepdims=True)
        return inputs_exp / inputs_sum


inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=10, output_dim=32, mask_zero=True)(inputs)
x = layers.Dense(1)(x)
outputs = TemporalSoftmax()(x)

model = keras.Model(inputs, outputs)
y = model(np.random.randint(0, 10, size=(32, 100)), np.random.random((32, 100, 1)))

### **总结**

以上就是你需要了解有关Keras中的填充和掩盖的全部信息，我们来回顾一下：

+ “掩盖”是各层知道如何跳过/忽略序列输入中的某些时间步长的方法。
+ 有些层是掩盖生成器：`Embedding`可以根据输入值（当`mask_zero=True`）生成掩盖，`Masking`层也同样可以。
+ 有些层是掩盖的使用者：它们在其`__call__`方法中暴露一个`mask`参数，如RNN层。
+ 在函数式API和Sequential API中，掩盖信息会自动传播。
+ 当以独立方式使用层时，可以将`mask`参数手动传递给层。
+ 你可以轻松地编写修改当前掩盖的层，生成新掩盖或使用与输入关联的掩盖的层。